## step03:
## Using the optimal estimated parameters to get the firing-rate for each subject in each trial for each choices

## Then simulate the BOLD signal

## loop for all participants

### 5 participants

In [1]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
import pandas as pd

from src.attractor_model import *


#import participants' data

subdata = pd.read_csv('data_PL_REWARD_020165_for_rstan.csv')



In [28]:
def get_firingrate(params):
    ######
    LR = params[0]

    #let xinyuan try the 1st participants
    

    #create the empty mat with nan to store value
    val = np.full([len(subdata_1),2], np.nan)
    val[0] = 0.5
    
    this_choice_data = np.zeros(len(subdata_1))
    
    for trial in range(0,len(subdata_1)-1):
        

    
        if subdata_1.accuracy.iloc[trial]==0:
            
            this_choice = subdata_1.inc_option.iloc[trial]
            
            this_choice_data[trial] = this_choice
            
            val[trial+1][this_choice-1]=val[trial][this_choice-1]+LR*(subdata_1.f_inc.iloc[trial]-val[trial][this_choice-1])
            val[trial+1][1-this_choice+1] = 1-val[trial+1][this_choice-1]
        if subdata_1.accuracy.iloc[trial]==1:
            this_choice = subdata_1.cor_option.iloc[trial]
            
            this_choice_data[trial] = this_choice
            
            val[trial+1][this_choice-1]=val[trial][this_choice-1]+LR*(subdata_1.f_cor.iloc[trial]-val[trial][this_choice-1])
            val[trial+1][1-this_choice+1] = 1-val[trial+1][this_choice-1]
    

#getting the value difference
    val_diff = []

    for kk in range(0,len(subdata_1)):
    
        thisval= val[kk][0]-val[kk][1]
   
        val_diff.append(thisval)

    #end loop here
    
    #print(len(val_diff))
    




    for cPrime in range(0,len(val_diff)):#trial by trial
        
        
        


        pars['coh'] = val_diff[cPrime]*10#to make the value not to be too small
        #print('coherence-:',pars['coh'])
        expt_pars['Ntrials'] = 1 # Total number of trials
        pars['JE'] = params[1]
        pars['JI'] = params[2]
        frThreshold = params[3]
        
    
        results = run_sim(pars, expt_pars, expt='DM')
        ##get the firing rate
        firingrate1 = results['r1smooth']#
        firingrate2 = results['r2smooth']#
        

    
        r1_val[cPrime] = results['r1smooth'][0]
        r2_val[cPrime] = results['r2smooth'][0]
        r_avg[cPrime] = (r1_val[0]+r2_val[0])/2

        S1_val[cPrime]= results['S1'][0]
        S2_val[cPrime]= results['S1'][0]
        S_avg[cPrime] = (S1_val[0]+S2_val[0])/2
    
    
        for r1smooth, r2smooth in zip(results['r1smooth'], results['r2smooth']): # loop over trials
            rt1 = next((t for t,r1 in zip(results['t'], r1smooth) if r1 > frThreshold), expt_pars['Ttotal'])
            rt2 = next((t for t,r2 in zip(results['t'], r2smooth) if r2 > frThreshold), expt_pars['Ttotal'])
            #print('rt1 is-',rt1)
            #print('rt2 is-',rt2)
            if rt1 < rt2:
                #allchoice.append(1)
                allRT_temp[cPrime]=float(rt1)
                allchoice[cPrime] = 1
              
            if rt2 < rt1:
                
                
                #allchoice.append(2)
                allRT_temp[cPrime]=float(rt2)
                allchoice[cPrime] = 2
               
                
            if rt2==rt1:
                                    
                                    
                allRT_temp[cPrime]=(1+np.random.rand(1)[0])
                allchoice[cPrime] = 0
                

            



    return allRT_temp,S1_val,S2_val,r1_val,r2_val,S_avg,r_avg,allchoice,this_choice_data


In [3]:
#parameters keep same with WANG's MODEL for economic decisions
def F(I, a=175., b=30., d=0.100):
    """F(I) for vector I"""
    
    return (a*I - b)/(1.-np.exp(-d*(a*I - b)))

def fbold(t, p=2., taub=1.25, o=2.25):
    """fbold(t) kernel for vector t"""
    kernel  = np.zeros_like(t)
    idx = t >= o
    kernel[idx] = ((t[idx] - o)/taub)**(p-1) / np.math.factorial(p-1) * np.exp(-(t[idx] - o)/taub)
    
    return kernel


def default_BOLDsim_pars(C,**kwargs):
    parsbold = {}
    # Network Parameters
    parsbold['C'] = 1           # structural connectivity
    parsbold['w'] = 0.55        # self-excitation scaling factor
    parsbold['G'] = 3.5         # long-range scaling factor
    parsbold['JN'] = 0.2609     # overall excitatory strength [nA]
    parsbold['I0'] = 0.3255       # 0.3255 # background current [nA]
    parsbold['tauS'] = 0.1      # Synaptic time constant [sec]
    parsbold['gamma'] = 0.641   # Saturation factor for gating variable
    parsbold['tau0'] = 0.002    # Noise time constant [sec]
    parsbold['sigma'] = 0.02   # Noise magnitude [nA]
    # Simulation Parameters
    parsbold['N'] = 1  # number of nodes
    parsbold['T'] = 3.5         # Total duration of simulation [sec]
    parsbold['dt'] = 0.0005      # Simulation time step [sec]

    for k in kwargs:
        parsbold[k] = kwargs[k]
    
    parsbold['range_t'] = np.arange(0, parsbold['T'], parsbold['dt']) # Vector of time points 
        
    return parsbold



def run_simBOLD(parsbold,Sinput,rinput):
    """S(t), r(t), Isyn(t), Ieta(t), b(t) for vector t"""
   
    # Retrieve parameters
    C = parsbold['C']
    N = parsbold['N']
    w, G = parsbold['w'], parsbold['G'] 
    JN = parsbold['JN']
    tauS, gamma = parsbold['tauS'], parsbold['gamma']
    I0, tau0, sigma = parsbold['I0'], parsbold['tau0'], parsbold['sigma']
    
    range_t = parsbold['range_t']
    nt = range_t.size
    dt = range_t[1] - range_t[0]
    
    S = Sinput
    
    # Initialize
    if (np.shape(S)[0]==0):
        S = np.zeros([N, nt])#np.zeros([N, nt])
    #else:
        # continue simulation with previous state
        #S[:,0]= S[:,-1]#+np.random.randn(N)*1e-2

    B = np.zeros([N, nt])
    #r = np.zeros([N, nt])
    r = rinput
    Isyn = np.zeros(N)#np.zeros([N, nt])
    

    
    # Initialize OU process
    Ieta = I0 + sigma*np.random.randn(N)
    noise = np.random.randn(N,nt)
    verbose = False
    if verbose:
        print('Running...')
        progress = 0
    for idx in range(1, nt):
    
        # Total synaptic input
        Isyn = w*JN*S[idx-1] + G*JN*C*S[idx-1] + Ieta

        # Transfer function to get firing rate
        #r[:, idx]  = F(Isyn)
    
        #---- Dynamical equations -------------------------------------------
    
        # Update Mean NMDA-mediated synaptic dynamics
        S[idx] = S[idx-1] + dt*(-S[idx-1]/tauS + (1-S[idx-1])*gamma*r[idx])
    
        # Update Ornstein-Uhlenbeck process
        Ieta = Ieta + (dt/tau0)*(I0-Ieta) + np.sqrt(dt/tau0)*sigma*noise[:,idx]

        if verbose:
            if 100.*(idx+1)/nt - progress >= 10:
                progress += 10
                print('\r',f'{progress:4d} %', end='')
    if verbose:
        print('')
    
    # Extract BOLD signal
    if verbose:
        print('extracting BOLD signal ...')
    fbold_vec = fbold(range_t)
    #for idx in range(N):
    B = np.convolve(S, fbold_vec)
    
    if verbose:
        print('done!')
    z = {'S':S, 'r':r, 'Isyn': Isyn, 'Ieta':Ieta, 'B': B}
    return z

In [29]:
for whichsub in range(1,6):
    #load data
    subdata_1= subdata[(subdata.participant==1)]

    r1_val = np.zeros([len(subdata_1),7000])
    r2_val = np.zeros([len(subdata_1),7000])
    r_avg = np.zeros([len(subdata_1),7000])

    S1_val = np.zeros([len(subdata_1),7001])
    S2_val = np.zeros([len(subdata_1),7001])    
    S_avg = np.zeros([len(subdata_1),7001])
    allRT_temp = np.zeros(len(subdata_1))

    allchoice = np.zeros(len(subdata_1))


    pars = default_pars()
    expt_pars = default_expt_pars()
    
    df_allsub_para = pd.read_csv('allsub_para.csv')

    thissub_para = np.zeros([4])
    thissub_para[0] = df_allsub_para['learning-rate'][whichsub-1]
    thissub_para[1] = df_allsub_para['JE'][whichsub-1]
    thissub_para[2] = df_allsub_para['JI'][whichsub-1]
    thissub_para[3] = df_allsub_para['threshold'][whichsub-1]
    
        #subdata_1= subdata[(subdata.participant==1)]
        

    rt_data = subdata_1['rt']
    #allRT_temp = np.zeros(len(subdata_1))
    allRT_temp,S1_val,S2_val,r1_val,r2_val,S_avg,r_avg,subchoice,this_choice_data = get_firingrate(thissub_para)





    #  get default parameters
    default_BOLDparams=default_BOLDsim_pars(1,)
    paramsBOLD=default_BOLDparams.copy()
    #paramsBOLD['G']=1.0

    # run simulation

    bold1 = np.zeros([len(subdata_1),7000])
    mean_bold1 = np.zeros(len(subdata_1))

    bold2 = np.zeros([len(subdata_1),7000])
    mean_bold2 = np.zeros(len(subdata_1))


    for kk in range(0,len(subdata_1)):

        S_input = S1_val[kk]
        R_input = r1_val[kk]
        DATAPACK1 = run_simBOLD(paramsBOLD,S_input,R_input)


        #print(DATAPACK['B'][kk])
        bold1[kk]  = DATAPACK1['B'][0:7000]
        mean_bold1[kk]  = np.mean(DATAPACK1['B'][0:7000])


        #print(DATAPACK['B'])
        #DATAPACK['B']



    for kk in range(0,len(subdata_1)):

        S_input = S2_val[kk]
        R_input = r2_val[kk]
        DATAPACK2 = run_simBOLD(paramsBOLD,S_input,R_input)


        #print(DATAPACK['B'][kk])
        bold2[kk]  = DATAPACK2['B'][0:7000]
        mean_bold2[kk]  = np.mean(DATAPACK2['B'][0:7000])


    # show the choice-based bold sigmal

    sub_meanbold = np.zeros(len(subdata_1))

    for kk in range(0,len(subdata_1)):

        if subchoice[kk]==1:
            sub_meanbold[kk] = mean_bold1[kk]

        if subchoice[kk]==2:
            sub_meanbold[kk] = mean_bold2[kk]
    
    
    subRT_choice = np.zeros([len(subdata_1),5])
    subRT_choice[:,0] = allRT_temp
    subRT_choice[:,1] = subchoice
    subRT_choice[:,2] = rt_data
    choice_data = this_choice_data
    subRT_choice[:,3] = choice_data
    subRT_choice[:,4] = sub_meanbold




    df_sub_meanbold = pd.DataFrame(subRT_choice)
    df_sub_meanbold.columns = ['modeled_RT','modeled_choice','RT','choice','meanbold']  
    #show that
    #df_sub_meanbold

    df_sub_meanbold.to_csv(r'sub_'+str(whichsub)+'_alldata.csv')



In [20]:
    
    
    subRT_choice = np.hstack([allRT_temp,subchoice])



In [21]:
pd.DataFrame(subRT_choice)

,0
0,1.7010
1,1.4435
2,1.6425
3,1.4445
4,1.4415
...,...
425,2.0000
426,1.0000
427,1.0000
428,1.0000


In [22]:
    subRT_choice = np.zeros([len(subdata_1),2])
    rt_data = subdata_1['rt']

    subRT_choice[:,0] = allRT_temp
    subRT_choice[:,1] = subchoice



In [23]:
pd.DataFrame(subRT_choice)

,0,1
0,1.7010,2.0
1,1.4435,2.0
2,1.6425,1.0
3,1.4445,2.0
4,1.4415,2.0
...,...,...
210,1.6610,2.0
211,1.6400,1.0
212,1.5540,1.0
213,1.5440,1.0
